In [10]:
import os
import networkx as nx
import json
import plotly.graph_objects as go
import webbrowser

In [ ]:
def draw_interactive_graph():
    script_dir = os.getcwd()
    graph_dir = os.path.join(script_dir, 'children_data') 
    output_dir = os.path.join(script_dir, 'graph')

    if not os.path.exists(graph_dir):
        os.makedirs(graph_dir)

    for folder in os.listdir(graph_dir):
        folder_path = os.path.join(graph_dir, folder)
        if os.path.isdir(folder_path):
            # creating graph
            G = nx.DiGraph()
            parent = folder
            G.add_node(parent)
            for file in os.listdir(f'{graph_dir}/{folder}'):
                first_child = file.split('.')[0]
                G.add_node(first_child)
                G.add_edge(parent, first_child)
                with open(f'{graph_dir}/{folder}/{file}', 'r') as f:
                    if not file.endswith('_error.json'):
                        children = json.load(f)
                        if children:
                            for child in children:
                                if type(child) == str:
                                    print(children)
                                for celex_num, url in child.items():
                                    G.add_node(celex_num)
                                    G.add_edge(first_child, celex_num)

            pos = nx.spring_layout(G)
            for node in G.nodes():
                G.nodes[node]['pos'] = list(pos[node])

            edge_x = []
            edge_y = []
            for edge in G.edges():
                x0, y0 = G.nodes[edge[0]]['pos']
                x1, y1 = G.nodes[edge[1]]['pos']
                edge_x.append(x0)
                edge_x.append(x1)
                edge_x.append(None)
                edge_y.append(y0)
                edge_y.append(y1)
                edge_y.append(None)

            edge_trace = go.Scatter(
                x=edge_x, y=edge_y,
                line=dict(width=0.5, color='#888'),
                hoverinfo='none',
                mode='lines')

            node_x = []
            node_y = []
            for node in G.nodes():
                x, y = G.nodes[node]['pos']
                node_x.append(x)
                node_y.append(y)

            node_trace = go.Scatter(
                x=node_x, y=node_y,
                mode='markers+text',
                text=[node for node in G.nodes()],
                textposition="top center",
                hoverinfo='text',
                marker=dict(
                    showscale=True,
                    colorscale='YlGnBu',
                    size=10,
                    colorbar=dict(
                        thickness=15,
                        title='Node Connections',
                        xanchor='left',
                        # titleside='right'
                    ),
                    line_width=2))

            fig = go.Figure(data=[edge_trace, node_trace],
                            layout=go.Layout(
                                title=f'Interactive Graph for {parent}',
                                # titlefont_size=16,
                                showlegend=False,
                                hovermode='closest',
                                margin=dict(b=20, l=5, r=5, t=40),
                                annotations=[dict(
                                    text="Interactive Graph",
                                    showarrow=False,
                                    xref="paper", yref="paper",
                                    x=0.005, y=-0.002)],
                                xaxis=dict(showgrid=False, zeroline=False),
                                yaxis=dict(showgrid=False, zeroline=False))
                            )

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        output_path = os.path.join(output_dir, f'{parent}_interactive.html')
        fig.write_html(output_path)
        print(f'Interactive graph for {parent} created successfully!')
        webbrowser.open(f'file://{output_path}')


In [40]:
draw_interactive_graph()

['./../../../../legal-content/EN/AUTO/?uri=OJ:L:2015:123:TOC', './../../../../legal-content/EN/AUTO/?uri=OJ:L:2013:348:TOC']


AttributeError: 'str' object has no attribute 'items'